In [1]:
from eolearn.core import EOWorkflow, Dependency
from eolearn.core import FeatureType

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio.features
import rasterio.transform
import datetime
import os
from pathlib import Path
from math import ceil

In [3]:
# testing
import scipy.stats as stats

In [4]:
from tqdm import tqdm_notebook as tqdm

In [5]:
from eolearn.core import LoadFromDisk, SaveToDisk, AddFeature, EOPatch, EOTask, FeatureTypeSet, FeatureType, LinearWorkflow, EOExecutor
from eolearn.io import S2L1CWCSInput, AddSen2CorClassificationFeature
from eolearn.features import LinearInterpolation

In [6]:
from eolearn.mask import AddCloudMaskTask, get_s2_pixel_cloud_detector, AddValidDataMaskTask

/Users/markbogataj/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [10]:
buffer = -5
fraction_size = 0.05 # koliksen del podatkov vzame
spatial = False
window = 1 # če je 1 je per pixel, drugače pa glede na velikost okna
interpolation_range = [8]#2,4,8,16,32,64,128] # [2,4,8] # [16, 32] # [2,4,8,16,32]
mask_valid = 'VALID_DATA' # L1C_VALID changed from VALID_DATA
#resampled_range = ('2017-01-01', '2017-09-30', interpolation)
drop_classes = [1,3,4,5,8,14,16,18,19,21]
class_labels = set(list(range(1,26))) - set(drop_classes)
MAIN_FOLDER = Path('/Volumes/Seagate_drive') # spremenis na path do podatkov zip-a
DATA_FOLDER = os.path.join(MAIN_FOLDER, 'original')
DATA_LIST = os.listdir(DATA_FOLDER)
#OUTPUT_FOLDER = os.path.join(MAIN_FOLDER, 'interpolation-{}days'.format(interpolation))

In [11]:
actual_fraction = fraction_size/(window*window)
print(actual_fraction, actual_fraction*(window**2))

0.05 0.05


In [12]:
non_empty_patches =[]
for name in DATA_LIST:
    eopatch = EOPatch.load(os.path.join(DATA_FOLDER, name), lazy_loading=True)
    if len(eopatch.data)!=0:
        non_empty_patches.append(name)
DATA_LIST = non_empty_patches

In [15]:
#DATA_LIST= DATA_LIST[:1]
print(len(DATA_LIST))
DATA_LIST


213


['eopatch_100_col-8_row-7',
 'eopatch_101_col-8_row-8',
 'eopatch_102_col-8_row-9',
 'eopatch_103_col-8_row-10',
 'eopatch_107_col-9_row-3',
 'eopatch_108_col-9_row-4',
 'eopatch_109_col-9_row-5',
 'eopatch_10_col-1_row-7',
 'eopatch_110_col-9_row-6',
 'eopatch_111_col-9_row-7',
 'eopatch_112_col-9_row-8',
 'eopatch_113_col-9_row-9',
 'eopatch_114_col-9_row-10',
 'eopatch_119_col-10_row-3',
 'eopatch_120_col-10_row-4',
 'eopatch_121_col-10_row-5',
 'eopatch_122_col-10_row-6',
 'eopatch_123_col-10_row-7',
 'eopatch_124_col-10_row-8',
 'eopatch_125_col-10_row-9',
 'eopatch_126_col-10_row-10',
 'eopatch_127_col-10_row-11',
 'eopatch_12_col-1_row-9',
 'eopatch_130_col-11_row-1',
 'eopatch_131_col-11_row-2',
 'eopatch_132_col-11_row-3',
 'eopatch_133_col-11_row-4',
 'eopatch_134_col-11_row-5',
 'eopatch_135_col-11_row-6',
 'eopatch_136_col-11_row-7',
 'eopatch_137_col-11_row-8',
 'eopatch_138_col-11_row-9',
 'eopatch_139_col-11_row-10',
 'eopatch_13_col-1_row-10',
 'eopatch_140_col-11_row-1

In [16]:
class bufferPolys(EOTask):
    
    def __init__(self, features, buffer):
        self.feature_type, self.feature_name = features[0], features[1]
        self.buffer = buffer
    
    def execute(self, eop):
        # crops = eopatch.vector_timeless['LPIS_2017']
        crops = eop[self.feature_type][self.feature_name]
        crops_buffered = crops.copy(deep=True)
        # make buffer = -5
        crops_buffered.geometry = crops.buffer(self.buffer)
        crops_buffered = crops_buffered[~crops_buffered.is_empty]
        crops_buffered['SIFRA_KMRS'] = pd.to_numeric(crops_buffered['SIFRA_KMRS'])

        eop.add_feature(FeatureType.VECTOR_TIMELESS, 'LPIS_buffered', crops_buffered)
        
        return eop

In [17]:
class VectorToRasterMultiple(EOTask):
    """
    Task burns into one of the EOPatch's features geo-referenced shapes given in provided Geopandas DataFrame.

    :param feature: A tuple of feature type and feature name, e.g. (FeatureType.MASK, 'cloud_mask')
    :type feature: (FeatureType, str)
    :param vector_data: Vector data
    :type vector_data: geopandas.GeoDataFrame
    :param raster_value: Value of raster pixels which are contained inside of vector polygons
    :type raster_value: int or float
    :param raster_shape: Can be a tuple in form of (height, width) of an existing feature from which the shape will be
                            taken e.g. (FeatureType.MASK, 'IS_DATA')
    :type raster_shape: (int, int) or (FeatureType, str)
    :param raster_dtype: `numpy` data type of the obtained raster array
    :type raster_dtype: numpy.dtype
    :param no_data_value: Value of raster pixels which are outside of vector polygons
    :type no_data_value: int or float
    :param kwargs: arguments passed to rasterio.features.rasterize
    """
    def __init__(self, feature, vector_feature, raster_value, raster_shape, raster_dtype=np.uint8, no_data_value=0,
                 **kwargs):
        self.feature_type, self.feature_name = next(iter(self._parse_features(feature)))
        self.vector_feature = vector_feature
        self.raster_value = raster_value
        self.raster_shape = raster_shape
        self.raster_dtype = raster_dtype
        self.no_data_value = no_data_value
        self.kwargs = kwargs

    def _get_submap(self, eopatch):
        """
        Returns a new geopandas dataframe with same structure as original one (columns) except that
        it contains only polygons that are contained within the given bbox.

        :param eopatch: input EOPatch
        :type eopatch: EOPatch
        :return: New EOPatch
        :rtype: EOPatch
        """
        bbox_poly = eopatch.bbox.get_geometry()
        vector_data = eopatch[self.vector_feature[0]][self.vector_feature[1]]
        
        vector_data = vector_data[vector_data.geometry.intersects(bbox_poly)].copy(deep=True)
        
        vector_data.geometry = vector_data.geometry.buffer(0)
        vetor_data = vector_data[vector_data.geometry.is_valid]
        vetor_data = vector_data[~vector_data.geometry.is_empty]
        
        filtered_data = vector_data[vector_data.geometry.intersects(bbox_poly)].copy(deep=True)
        pairs = []
        for idx, row in filtered_data.iterrows():
            pairs.append((row.geometry, row[self.raster_value]))

        return pairs

    def _get_shape(self, eopatch):
        if isinstance(self.raster_shape, (tuple, list)) and len(self.raster_shape) == 2:
            if isinstance(self.raster_shape[0], int) and isinstance(self.raster_shape[1], int):
                return self.raster_shape

            feature_type, feature_name = next(self._parse_features(self.raster_shape)(eopatch))
            return eopatch.get_spatial_dimension(feature_type, feature_name)

        raise ValueError('Could not determine shape of the raster image')

    def execute(self, eopatch):
        """ Execute function which adds new vector layer to the EOPatch

        :param eopatch: input EOPatch
        :type eopatch: EOPatch
        :return: New EOPatch with added vector layer
        :rtype: EOPatch
        """
        # print(eopatch)
        bbox_map = self._get_submap(eopatch)
        height, width = self._get_shape(eopatch)
        data_transform = rasterio.transform.from_bounds(*eopatch.bbox, width=width, height=height)

        if self.feature_name in eopatch[self.feature_type]:
            raster = eopatch[self.feature_type][self.feature_name].squeeze()
        else:
            raster = np.ones((height, width), dtype=self.raster_dtype) * self.no_data_value

        if len(bbox_map):
            rasterio.features.rasterize(bbox_map, out=raster,
                                        transform=data_transform,
                                        dtype=self.raster_dtype,
                                        **self.kwargs)

        eopatch[self.feature_type][self.feature_name] = raster[..., np.newaxis]

        return eopatch

In [18]:
class select_bands(EOTask):
    
    def __init__(self, feature, new_feature_name, features):
        self.feature_type, self.feature_name = feature[0], feature[1]
        self.new_feature_name = new_feature_name
        self.features = features
    
    def execute(self, eop):
        bands = eop[self.feature_type][self.feature_name]
        bands_new = bands[:, :, :, self.features]
        eop.add_feature(self.feature_type, self.new_feature_name, bands_new)
        
        return eop
    
class NormalizedDifferenceIndex(EOTask):   
    """
    The tasks calculates user defined Normalised Difference Index (NDI) between two bands A and B as:
    NDI = (A-B)/(A+B).
    """
    def __init__(self, feature_name, band_a, band_b):
        self.feature_name = feature_name
        self.band_a_fetaure_name = band_a.split('/')[0]
        self.band_b_fetaure_name = band_b.split('/')[0]
        self.band_a_fetaure_idx = int(band_a.split('/')[-1])
        self.band_b_fetaure_idx = int(band_b.split('/')[-1])
        
    def execute(self, eopatch):
        band_a = eopatch.data[self.band_a_fetaure_name][..., self.band_a_fetaure_idx]
        band_b = eopatch.data[self.band_b_fetaure_name][..., self.band_b_fetaure_idx]
        
        ndi = (band_a - band_b) / (band_a  + band_b)
        
        eopatch.add_feature(FeatureType.DATA, self.feature_name, ndi[..., np.newaxis])
        
        return eopatch
    
class EuclideanNorm(EOTask):   
    """
    The tasks calculates Euclidian Norm of all bands within an array:
    norm = sqrt(sum_i Bi**2),
    where Bi are the individual bands within user-specified feature array.
    """
    def __init__(self, feature_name, in_feature_name):
        self.feature_name = feature_name
        self.in_feature_name = in_feature_name
    
    def execute(self, eopatch):
        arr = eopatch.data[self.in_feature_name]
        norm = np.sqrt(np.sum(arr**2, axis=-1))
        
        eopatch.add_feature(FeatureType.DATA, self.feature_name, norm[..., np.newaxis])
        return eopatch

class ConcatenateData(EOTask):
    """ Task to concatenate data arrays along the last dimension
    """
    def __init__(self, feature_name, feature_names_to_concatenate):
        self.feature_name = feature_name
        self.feature_names_to_concatenate = feature_names_to_concatenate

    def execute(self, eopatch):
        arrays = [eopatch.data[name] for name in self.feature_names_to_concatenate]

        eopatch.add_feature(FeatureType.DATA, self.feature_name, np.concatenate(arrays, axis=-1))

        return eopatch

In [19]:
def undersampleMajority(mask, rows, cols, fraction):
    sampled = np.random.rand(len(rows)) > (1.0 - fraction*2) # increase sample size by 2 to later reduce by 50%
    
    sampled_classes = mask[rows[sampled], cols[sampled]]
    # checking the distribution of mapped classes
    sampled_classes = np.array([get_numeric_group(get_group_id(str(x).zfill(3), lpis_to_group), group_to_numeric) for x in sampled_classes])
    values = np.unique(sampled_classes, return_counts=True)
    number_of_2 = values[1][np.where(values[0]==2)[0]][0]
    
    for location, x in enumerate(sampled_classes):
        if x in drop_classes:
            sampled_classes[location] = 0
            
    clean = sampled_classes==0
    sampled_classes = sampled_classes[~clean]
    
    sampled2 = np.random.rand(number_of_2) > (1.0 - 1/6) # we want only 1/6 of them all
        
    sub_sample = np.where(sampled_classes==2)[0][~sampled2]
    sampled[np.where(sampled==True)[0][sub_sample]]=False
    
    #sampled_classes = mask[rows[sampled], cols[sampled]]
    #sampled_classes = np.array([get_numeric_group(get_group_id(str(x).zfill(3), lpis_to_group), group_to_numeric) for x in sampled_classes])
    #values = np.unique(sampled_classes, return_counts=True)
    #distrib = np.round(values[1]/sum(values[1]), 6)
    #fig = plt.figure(figsize=(15, 15))
    #plt.bar(values[0], distrib, align='center')
    #plt.xticks(values[0], values[0]);
    
    return sampled

def sample(mask, rows, cols, fraction):
    sampled = np.random.rand(len(rows)) > (1.0 - fraction)
    return sampled

class SampleValid(EOTask):
    """
    The task samples pixels with a value in given timeless feature different from no valid data value.
    """

    def __init__(self, feature, fraction=1.0, no_data_value=0, sample_features=...):
        """ Task to sample pixels from a reference timeless raster mask, excluding a no valid data value

        :param feature:  Reference feature used to select points to be sampled
        :param fraction: Fraction of valid points to be sampled
        :param no_data_value: Value of non-valid points to be ignored
        """
        self.feature_type, self.feature_name, self.new_feature_name = next(
            self._parse_features(feature, new_names=True,
                                 default_feature_type=FeatureType.MASK_TIMELESS,
                                 allowed_feature_types={FeatureType.MASK_TIMELESS},
                                 rename_function='{}_SAMPLED'.format)())
        self.fraction = fraction
        self.no_data_value = no_data_value
        self.sample_features = self._parse_features(sample_features)

    def execute(self, in_eopatch, eopatch_folder, OUTPUT_FOLDER, seed=None):
        eopatch = in_eopatch.__copy__()
        #print(eopatch_folder)
        mask = eopatch[self.feature_type][self.feature_name].squeeze()

        if mask.ndim != 2:
            raise ValueError('Invalid shape of sampling reference map.')
        
        rows, cols = np.where(mask != self.no_data_value)
        sampling_file = Path(os.path.join(OUTPUT_FOLDER, eopatch_folder, 'sampling.npy'))
        
        np.random.seed(seed)
        if not os.path.exists(os.path.join(OUTPUT_FOLDER, eopatch_folder)):
            os.makedirs(os.path.join(OUTPUT_FOLDER, eopatch_folder))
            
        if sampling_file.is_file():
            sampled = np.load(sampling_file)
            if not(round(np.sum(sampled)/len(rows), 2) == self.fraction):
                print("fraction mismatch")
                #sampled = undersampleMajority(mask, rows, cols, self.fraction)
                sampled = sample(mask, rows, cols, self.fraction)
                np.save(sampling_file, sampled)
            
        else:
            #sampled = undersampleMajority(mask, rows, cols, self.fraction)            
            sampled = sample(mask, rows, cols, self.fraction) 
            print("saving new fraction, file didn't exist")
            np.save(sampling_file, sampled)
            
        rows = rows[sampled]
        cols = cols[sampled]

        for feature_type, feature_name in self.sample_features(eopatch):

            if feature_type in FeatureTypeSet.RASTER_TYPES.intersection(FeatureTypeSet.SPATIAL_TYPES):

                if feature_type.is_time_dependent():
                    sampled_data = eopatch[feature_type][feature_name][:, rows, cols, :]
                else:
                    sampled_data = eopatch[feature_type][feature_name][rows, cols, :]

                # here a copy of sampled array is returned and assigned to feature of a shallow copy
                # orig_eopatch[feature_type][feature_name] remains unmodified
                eopatch[feature_type][feature_name] = sampled_data[..., np.newaxis, :]

        new_mask = np.ones_like(mask)*self.no_data_value
        new_mask[rows, cols] = mask[rows, cols]        
        eopatch[self.feature_type][self.new_feature_name] = new_mask[..., np.newaxis]
        eopatch[FeatureType.SCALAR_TIMELESS]['SAMPLED_ROWS'] = rows
        eopatch[FeatureType.SCALAR_TIMELESS]['SAMPLED_COLS'] = cols

        return eopatch

In [22]:
class SampleValidSpatial(EOTask):
    """
    The task samples pixels with a value in given timeless feature different from no valid data value.
    """

    #def __init__(self, feature, fraction=1.0, window=24, no_data_value=0, increase=1, sample_features=...):
    def __init__(self, feature, fraction=1.0, window=1, no_data_value=0, increase=1, sample_features=...):
        """ Task to sample pixels from a reference timeless raster mask, excluding a no valid data value

        :param feature:  Reference feature used to select points to be sampled
        :param fraction: Fraction of valid points to be sampled
        :param no_data_value: Value of non-valid points to be ignored
        """
        self.feature_type, self.feature_name, self.new_feature_name = next(
            self._parse_features(feature, new_names=True,
                                 default_feature_type=FeatureType.MASK_TIMELESS,
                                 allowed_feature_types={FeatureType.MASK_TIMELESS},
                                 rename_function='{}_SAMPLED'.format)())
        self.fraction = fraction
        self.no_data_value = no_data_value
        self.sample_features = self._parse_features(sample_features)
        self.window = window
        self.increase = increase

    def sample_spatial(self, mask, rows, cols, fraction):

        # double the amount of sampled data to be reduced based on coverage
        sampled = np.random.rand(len(rows)) > (1.0 - self.fraction*self.increase)
        #print("sampled", np.sum(sampled), "which is", np.sum(sampled)/len(rows))
        # TODO save sampling        
        rows = rows[sampled]
        cols = cols[sampled]

        new_mask = np.array([mask[(x-self.window//2):(x+ceil(self.window/2)), (y-self.window//2):(y+ceil(self.window/2))] for x,y in zip(rows,cols)])
        no_data_part = []        
        # to increase and reduce data where there is data
        if self.increase != 1:
            for x in new_mask:
                uniq = np.unique(x, return_counts=True)
                if self.no_data_value in uniq[0]:
                    number_of_0 = uniq[1][np.where(uniq[0]==self.no_data_value)[0]][0]
                    no_data_part.append(number_of_0/np.sum(uniq[1]))
                else: # in case all samples have label different from no_data_value
                    no_data_part.append(0)

            ordered = np.array(sorted(enumerate(no_data_part), key=lambda x:x[1]))
            to_keep = np.array(ordered[:(len(ordered)//self.increase),0], dtype= np.int16)
            #print(len(to_keep))
            new_mask = new_mask[to_keep]
            rows = rows[to_keep]
            cols = cols[to_keep]
            
            # testing distribution
            #print(np.unique(np.round(np.array(no_data_part)[to_keep],1), return_counts=True))
            # print(np.unique(np.round(no_data_part,1), return_counts=True))
            # fit = stats.norm.pdf(ordered[:,1], np.mean(ordered[:,1]), np.std(ordered[:,1]))
            # plt.plot(ordered[:,1], fit, '-')
            # plt.hist(ordered[:,1], normed=True)
            # plt.savefig("dist1.png")
            # #print(ordered.shape)
            # #print(ordered)

            #-----------testing distribution of no data
            # no_data_part = []
            # n_pixels=0
            # for x in new_mask:
            #     uniq = np.unique(x, return_counts=True)
            #     if self.no_data_value in uniq[0]:
            #         number_of_0 = uniq[1][np.where(uniq[0]==self.no_data_value)[0]][0]
            #         no_data_part.append(number_of_0/np.sum(uniq[1]))
            #     else: # in case all samples have label different from no_data_value
            #         no_data_part.append(0)

            # ordered = np.array(sorted(enumerate(no_data_part), key=lambda x:x[1]))

            # print(np.unique(np.round(no_data_part,1), return_counts=True))
            # fit = stats.norm.pdf(no_data_part, np.mean(no_data_part), np.std(no_data_part))
            # plt.plot(no_data_part, fit, '-')
            # plt.hist(no_data_part, normed=True)
            # plt.savefig("dist2.png")
            #print(len(ordered))
            #print(ordered)
        
        return rows, cols
       
        
    
    def execute(self, in_eopatch, eopatch_folder, OUTPUT_FOLDER, seed=None):
        eopatch = in_eopatch.__copy__()
        
      

        
        # new_mask = np.array([mask[(x-self.window//2):(x+ceil(self.window/2)), (y-self.window//2):(y+ceil(self.window/2))] for x,y in zip(rows,cols)])
        #print(mask.shape)
        #print(new_mask.shape)
        # reducing data based on no_data_label
        
        # no_data_part = []        
        # to increase and reduce data where there is data
        # if increase != 1:
        #     for x in new_mask:
        #         uniq = np.unique(x, return_counts=True)
        #         if self.no_data_value in uniq[0]:
        #             number_of_0 = uniq[1][np.where(uniq[0]==self.no_data_value)[0]][0]
        #             no_data_part.append(number_of_0/np.sum(uniq[1]))
        #         else: # in case all samples have label different from no_data_value
        #             no_data_part.append(0)
        #         
        #     ordered = np.array(sorted(enumerate(no_data_part), key=lambda x:x[1]))
        #     to_keep = np.array(ordered[:(len(ordered)//increase),0], dtype= np.int16)
        #     #print(len(to_keep))
        #     new_mask = new_mask[to_keep]
        #     rows = rows[to_keep]
        #     cols = cols[to_keep]
        
        
        
        #-----------testing distribution of no data
        #no_data_part = []
        #n_pixels=0
        #for x in new_mask:
        #    uniq = np.unique(x, return_counts=True)
        #    number_of_0 = uniq[1][np.where(uniq[0]==self.no_data_value)[0]][0]
        #    no_data_part.append(number_of_0/np.sum(uniq[1]))
        #    n_pixels += np.sum(uniq[1])
        #    #print(uniq)
        ##print(n_pixels)
        #ordered = sorted(no_data_part)
        #fit = stats.norm.pdf(ordered, np.mean(ordered), np.std(ordered))
        #plt.plot(ordered, fit, '-')
        #plt.hist(ordered, normed=True)
        #plt.show()

        
        
         # ------------------from sampling valid
        mask = eopatch[self.feature_type][self.feature_name].squeeze()
        h, w = mask.shape
        if mask.ndim != 2:
            raise ValueError('Invalid shape of sampling reference map.')

        np.random.seed(int(eopatch_folder.split('_')[1]))
        increase = 1
        rows1, cols1 = np.where(mask != self.no_data_value)
        # moving towards the inside of the patch so patch is defined in sampled area
        clean = np.logical_and(rows1>=self.window//2, rows1<=(h-ceil(self.window/2)))
        # #print("bounds x", len(rows)-np.sum(clean))
        rows1 = rows1[clean]
        cols1 = cols1[clean]
        clean = np.logical_and(cols1>=self.window//2, cols1<=(w-ceil(self.window/2)))
        #print("bounds y", len(cols)-np.sum(clean))
        rows1 = rows1[clean]
        cols1 = cols1[clean]
        

               
        sampling_file = Path(os.path.join(OUTPUT_FOLDER, eopatch_folder+'sampling.npy'))
        np.random.seed(seed)
        if not os.path.exists(OUTPUT_FOLDER):
            os.makedirs(OUTPUT_FOLDER)
            
        if sampling_file.is_file(): # not loading fraction beacause just one interpolation sampling_file.is_file():
            rows, cols = np.load(sampling_file, allow_pickle=True)
            if not(round(len(rows)/len(rows1), 3) == self.fraction):
                print("fraction mismatch", round(len(rows)/len(rows1), 3), self.fraction)
                #rows, cols = self.sample_spatial(mask, rows1, cols1, self.fraction)
                #np.save(sampling_file, (rows, cols))
            
        else:
            rows, cols = self.sample_spatial(mask, rows1, cols1, self.fraction) 
            print("saving new fraction, file didn't exist")
            np.save(sampling_file, (rows, cols))
            
        #rows = rows[sampled]
        #cols = cols[sampled] #empty
        
        for feature_type, feature_name in self.sample_features(eopatch):
            if feature_type in FeatureTypeSet.RASTER_TYPES.intersection(FeatureTypeSet.SPATIAL_TYPES):
                if feature_type.is_time_dependent():
                    sampled_data = np.array([eopatch[feature_type][feature_name][:,(x-self.window//2):(x+ceil(self.window/2)), (y-self.window//2):(y+ceil(self.window/2)), :] for x,y in zip(rows,cols)])
                    sampled_data = np.transpose(sampled_data, (1, 0, 2, 3, 4))
                    t,s,wi,he,b = sampled_data.shape
                    sampled_data = sampled_data.reshape(t,s,wi*he,b)
                else:
                    sampled_data = np.array([eopatch[feature_type][feature_name][(x-self.window//2):(x+ceil(self.window/2)), (y-self.window//2):(y+ceil(self.window/2)), :] for x,y in zip(rows,cols)])
                    s,wi,he,b = sampled_data.shape
                    sampled_data = sampled_data.reshape(s,wi*he,b)
        
                eopatch[feature_type][feature_name] = sampled_data
        
        #print(list(zip(rows, cols))[:10])
        #print(mask[rows[:10], cols[:10]])
        new_mask = np.ones_like(mask)*self.no_data_value
        new_mask[rows, cols] = mask[rows, cols]
        eopatch[self.feature_type][self.new_feature_name] = new_mask[..., np.newaxis]
        eopatch[FeatureType.SCALAR_TIMELESS]['SAMPLED_ROWS'] = rows
        eopatch[FeatureType.SCALAR_TIMELESS]['SAMPLED_COLS'] = cols

        return eopatch

In [21]:
def get_group_id(crop_group, crop_group_df, group_name='CROP_ID',
             group_id='GROUP_1', default_value=0):
    """
    Returns numeric crop group value for specified crop group name. The mapping is obtained from
    the specified crop group pandas DataFrame.
    """
    values = crop_group_df[crop_group_df[group_name]==crop_group][group_id].values
    if len(values)==0:
        return default_value
    else:
        return values[-1]

def get_numeric_group(crop_group, crop_group_df, group_name='GROUP_1_NAME',
                 group_id='GROUP_1_ID', default_value=0):
    """
    Returns numeric crop group value for specified crop group name. The mapping is obtained from
    the specified crop group pandas DataFrame.
    """
    values = crop_group_df[crop_group_df[group_name]==crop_group][group_id].values
    if len(values)==0:
        return default_value
    else:
        return values[-1]

class group_crops_spatial(EOTask):
    def __init__(self, lpis_to_group, group_to_numeric):
        self.lpis_to_group = lpis_to_group
        self.group_to_numeric = group_to_numeric
    
    def execute(self, eop):
        arr = []
        if eop.mask_timeless['LPIS_sifra'].shape[1] == 1:
               arr = [[get_numeric_group(get_group_id(str(y).zfill(3), self.lpis_to_group), self.group_to_numeric)] for y in eop.mask_timeless['LPIS_sifra'].squeeze()]
        else: 
            for y in eop.mask_timeless['LPIS_sifra'].squeeze():
                arr.append([get_numeric_group(get_group_id(str(x).zfill(3), self.lpis_to_group), self.group_to_numeric) for x in y])
                    #print(np.unique(arr[-1], return_counts=True))
                #break
        #print(eop.mask_timeless['LPIS_sifra'].squeeze()[np.where(np.array(arr) == 23)])
        arr = np.array(arr)
        #aar shape[1] is 1 when no spatial data is included
        eop.mask_timeless['LPIS_sifra'] = arr[...,np.newaxis]
        #print(np.unique(arr, return_counts=True))
        return eop
    
class group_crops(EOTask):
    def __init__(self, lpis_to_group, group_to_numeric):
        self.lpis_to_group = lpis_to_group
        self.group_to_numeric = group_to_numeric
    
    def execute(self, eop):
        # print(np.unique(eop.mask_timeless['LPIS_sifra'], return_counts=True))
        groups = list(get_group_id(str(x).zfill(3), self.lpis_to_group) for x in eop.mask_timeless['LPIS_sifra'].squeeze())
        # print(np.unique(groups, return_counts=True))
        tt = np.array([get_numeric_group(x, self.group_to_numeric) for x in groups])
             # np.array(list(get_numeric_group(get_group_id(str(x).zfill(3), lpis_to_group), group_to_numeric) for x in sampled.mask_timeless['LPIS_sifra']))
        # print(np.unique(tt, return_counts=True))
        eop.mask_timeless['LPIS_sifra'] = tt.reshape([len(tt),1,1])
        # eop.data['BANDS-S2-L2A'] = np.squeeze(eop.data['BANDS-S2-L2A'], axis=2)
        # eop.mask_timeless['PID'] = np.squeeze(eop.mask_timeless['PID'])
        return eop

In [23]:
buffer_poly = bufferPolys((FeatureType.VECTOR_TIMELESS, 'LPIS_2017_org'), buffer)

load = LoadFromDisk(folder=DATA_FOLDER, lazy_loading=True)


vec_to_ras_lpis = VectorToRasterMultiple((FeatureType.MASK_TIMELESS, 'LPIS_sifra'),
                                    (FeatureType.VECTOR_TIMELESS, 'LPIS_buffered'),
                                    'SIFRA_KMRS', raster_shape=(FeatureType.MASK, 'IS_DATA'),
                                    raster_dtype=np.uint16, no_data_value=0)

vec_to_ras_pid = VectorToRasterMultiple((FeatureType.MASK_TIMELESS, 'PID'),
                                    (FeatureType.VECTOR_TIMELESS, 'LPIS_buffered'),
                                    'GERK_PID', raster_shape=(FeatureType.MASK, 'IS_DATA'),
                                    raster_dtype=np.uint32, no_data_value=0)


# [(FeatureType.DATA, 'BANDS-S2-L2A'), (FeatureType.MASK, mask_valid), FeatureType.MASK_TIMELESS])
sampling = SampleValidSpatial((FeatureType.MASK_TIMELESS, 'LPIS_sifra'),
                       fraction=actual_fraction, no_data_value=0,
                                window=window, increase=2,
                       sample_features=[(FeatureType.DATA, 'BANDS-S2-L2A'),
                                        (FeatureType.MASK, mask_valid),
                                        FeatureType.MASK_TIMELESS])

#B(B02), G(B03), R(B04), NIR (B08)
custom_bands = [2, 3, 4, 8]
select = select_bands((FeatureType.DATA, 'BANDS-S2-L2A'), 'BANDS', custom_bands)

# TASKS FOR CALCULATING NEW FEATURES
# NDVI: (B08 - B04)/(B08 + B04)
# NDWI: (B03 - B08)/(B03 + B08)
# NORM: sqrt(B02^2 + B03^2 + B04^2 + B08^2 + B11^2 + B12^2)
ndvi = NormalizedDifferenceIndex('NDVI', 'BANDS/3', 'BANDS/2')
ndwi = NormalizedDifferenceIndex('NDWI', 'BANDS/1', 'BANDS/3')
norm = EuclideanNorm('NORM','BANDS')

concatenate = ConcatenateData('FEATURES', ['BANDS', 'NDVI', 'NDWI', 'NORM'])

lpis_to_group = pd.read_csv(os.path.join(MAIN_FOLDER,"crop-definitions/slo_lpis_crop_to_group_mapping_20190517.csv"))
group_to_numeric = pd.read_csv(os.path.join(MAIN_FOLDER,"crop-definitions/crop_group_1_definition_20190517.csv"))
grouping = group_crops_spatial(lpis_to_group, group_to_numeric)

#if spatial:
#    sampling = SampleValidSpatial((FeatureType.MASK_TIMELESS, 'LPIS_sifra'),
#                                 fraction=fraction_size, no_data_value=0, window=window,
#                                  sample_features=[(FeatureType.DATA, 'BANDS-S2-L2A'),
#                                                   (FeatureType.MASK, mask_valid), FeatureType.MASK_TIMELESS])
#    # TODO test group_crops(lpis_to_group, group_to_numeric)
#    grouping = group_crops_spatial(lpis_to_group, group_to_numeric)
#
#else:
#    sampling = SampleValid((FeatureType.MASK_TIMELESS, 'LPIS_sifra'),
#                                 fraction=fraction_size, no_data_value=0, sample_features=[(FeatureType.DATA, 'BANDS-S2-L2A'), (FeatureType.MASK, mask_valid), FeatureType.MASK_TIMELESS])
#    grouping = group_crops(lpis_to_group, group_to_numeric)


/Users/markbogataj/.local/lib/python3.6/site-packages/eolearn/core/core_tasks.py:166: DeprecationWarning: This task is deprecated, use LoadTask instead
  warnings.warn('This task is deprecated, use LoadTask instead', DeprecationWarning)


FileNotFoundError: [Errno 2] File b'/Volumes/Seagate_drive/crop-definitions/slo_lpis_crop_to_group_mapping_20190517.csv' does not exist: b'/Volumes/Seagate_drive/crop-definitions/slo_lpis_crop_to_group_mapping_20190517.csv'

In [18]:
def eojob(i):
    # eopatch = eopatch_names[i]
    # print(i)
    result = workflow.execute(i)
        #{load: {'eopatch_folder': eopatch}, 
                               #validate_gbm: {'labels': classifier1.classes_},
                               #save: {'eopatch_folder': eopatch}
                              #})
    del result

# save for different interpolations
print(str(window) +'_' + str(int(actual_fraction*(window**2)*100)).zfill(2))
for interpolation in interpolation_range[::-1]:
    resampled_range = ('2017-01-01', '2017-09-30', interpolation)
    OUTPUT_FOLDER = os.path.join("/Volumes/Seagate_drive", 'aFractionIndices{}'.format(str(window)+'_'+str(int(actual_fraction*(window**2)*100)).zfill(2)),  'interpolation-{}days'.format(interpolation))
    
    # workflow
    linear_interp = LinearInterpolation(
        (FeatureType.DATA,'FEATURES'), # 'BANDS-S2-L2A'
        mask_feature=(FeatureType.MASK, mask_valid), # mask to be used in interpolation TODO change from VALID_DATA
        copy_features=[(FeatureType.MASK_TIMELESS, 'LPIS_sifra'), (FeatureType.MASK_TIMELESS, 'PID')], # features to keep
        resample_range=resampled_range, # set the resampling range
        bounds_error=False # extrapolate with NaN's
    )

    save = SaveToDisk(OUTPUT_FOLDER, overwrite_permission = 1)
    exec_params = []
    for name in DATA_LIST:  # [filename for filename in DATA_LIST if 'col-21_row-20' in filename]:
        #name = filename.rsplit('/', 1)[1]
        exec_params.append({
            load: dict(eopatch_folder=name),
            save: dict(eopatch_folder=name),
            sampling: dict(eopatch_folder=name, OUTPUT_FOLDER=OUTPUT_FOLDER)
        })
    
    workflow = LinearWorkflow(load, buffer_poly, vec_to_ras_lpis, vec_to_ras_pid, sampling, select, ndvi, ndwi, norm, concatenate, linear_interp, grouping, save)
    executor = EOExecutor(workflow, exec_params, save_logs=True)
    print("Working on", interpolation, "days,", "saving to", OUTPUT_FOLDER)
    executor.run(workers=16)
    #workflow.execute(exec_params[0])

24_05
Working on 8 days, saving to /home/jovyan/work/data/matejR/aFractionIndices24_05/interpolation-8days
fraction mismatch 0.0 8.680555555555556e-05


In [ ]:
# executor.make_report()